In [1]:
! pip install -q transformers
! pip install -q datasets
! pip install -q -U transformers tokenizers
!pip install -q rusenttokenize

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.1 requires transformers<4.17,>=4.1, but you have transformers 4.17.0 which is incompatible.


In [2]:
# ! pip install gdown

In [3]:
# !gdown --id 1-5ebQom0-51wMERqfPdUdvEgtAYXMvL1
# !gdown --id 1-AJAoivJlmpPoaJDRpMboLdSeyddVT9h
# !gdown --id 1-Cf7es8FrkSpXf99ZQzHwM7JhKLS8d0p

In [4]:
import io
import os
import torch
import numpy as np
from functools import partial
from tqdm.notebook import tqdm
from datasets import load_metric, load_dataset, Dataset
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          AutoModel,
                          GPT2Config,
                          BertConfig,
                          EvalPrediction,
                          DataCollatorWithPadding,
                          AutoTokenizer)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

set_seed(123)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# MODEL_NAME = 'sberbank-ai/rugpt3small_based_on_gpt2'
MODEL_NAME = 'bert-base-multilingual-cased'
labels_ids = {'H':0, 'M':1}
N_LABELS = len(labels_ids)

In [5]:
cd /kaggle/input/feats-ruatd/

/kaggle/input/feats-ruatd


In [6]:
data = load_dataset('csv', data_files={'train':  'train_feat.csv',
                                       'validation': 'val_feat.csv'}).shuffle()

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-654ae0a783d2d040/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(set(data['train']['Class'])))

LabelEncoder()

In [8]:
from rusenttokenize import ru_sent_tokenize

def sent_tokenize(text):
    sents = ru_sent_tokenize(text)
    if len(sents) != 1:
        return ' '.join((sents[0], sents[-1]))
    else:
        return sents[0]

In [9]:
def left_truncate(tokens):
    for key in tokens:
        tokens[key] = list(map(lambda x: x[-512:], tokens[key]))
    return tokens

In [10]:
def preprocess_examples(examples, features, tokenizer):
#     result = tokenizer(list(map(sent_tokenize, examples['Text'])), padding=False, truncation=True)
    result = left_truncate(tokenizer(examples['Text'], padding=False))
    feats = []
    for col in ['flesch_kincaid_grade', 'flesch_reading_easy', 
                'coleman_liau_index', 'automated_readability_index', 
                'lix', 'dttr', 'mtld', 'mamtld', 'simpson_index', 'hapax_index']:
        feats.append(examples[col])
    result['features'] = np.array(feats).T
    result["labels"] = le.transform(examples['Class'])
    return result

In [11]:
# model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=MODEL_NAME, num_labels=N_LABELS)
model_config = BertConfig.from_pretrained(pretrained_model_name_or_path=MODEL_NAME, num_labels=N_LABELS)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=MODEL_NAME, config=model_config)
# default to left padding
# tokenizer.padding_side = "left"
# # Define PAD Token = EOS Token = 50256
# tokenizer.pad_token = tokenizer.eos_token
# model_config.pad_token_id = model_config.eos_token_id

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [12]:
features = data['train'].column_names[4:]
tokenized_splits = data.map(partial(preprocess_examples, features=features, tokenizer=tokenizer), batched=True, remove_columns=data['train'].column_names)

  0%|          | 0/130 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/22 [00:00<?, ?ba/s]

In [13]:
tokenized_splits

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'features', 'labels'],
        num_rows: 129066
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'features', 'labels'],
        num_rows: 21511
    })
})

In [14]:
train_dataset = tokenized_splits['train'] #.select(range(50000))
eval_dataset = tokenized_splits['validation']#.select(range(1000))

In [15]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [16]:
import numpy as np

ACC = load_metric('accuracy')
F1 = load_metric('f1')

def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    preds = np.argmax(preds, axis=1)

    acc = ACC.compute(predictions=preds, references=p.label_ids)['accuracy']
    micro_f1 = F1.compute(predictions=preds, references=p.label_ids,
                              average='micro')['f1']
    macro_f1 = F1.compute(predictions=preds, references=p.label_ids,
                              average='macro')['f1']
    return {'acc': acc, 'micro_f1': micro_f1, 'macro_f1': macro_f1}

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

In [17]:
from torch import nn


class GPTClassifier(nn.Module):
    def __init__(self, model_config, n_classes):
        super().__init__()
        self.n_classes = n_classes
        # model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=MODEL_NAME, num_labels=N_LABELS)
        # model_config.pad_token_id = model_config.eos_token_id
        self.gpt = AutoModel.from_pretrained(MODEL_NAME, config=model_config)
        self.drop = nn.Dropout(p=0.5)
        self.f1 = nn.Linear(self.gpt.config.hidden_size, self.gpt.config.hidden_size)
        self.f2 = nn.Linear(40, 128)
        self.out = nn.Linear(self.gpt.config.hidden_size+128, n_classes)
  
    def forward(self, input_ids, attention_mask, features, labels=None):
        last_hidden_state, _ = self.gpt(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False)
        
        batch_size, sequence_length = input_ids.shape[:2]
        sequence_lengths = torch.ne(input_ids, self.gpt.config.pad_token_id).sum(-1) - 1
        pooled_output = self.f1(self.drop(last_hidden_state))
        feats = self.f2(features)

        pooled_output = pooled_output[torch.arange(batch_size), sequence_lengths]
        stacked_features = torch.hstack([pooled_output, feats])
        logits = self.out(self.drop(stacked_features))
       
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.n_classes), labels.view(-1))

        output = (logits,)
        return ((loss,) + output) if loss is not None else output

class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.5)
        self.f1 = nn.Linear(self.bert.config.hidden_size, 128)
        self.f2 = nn.Linear(10, 64)
        self.out = nn.Linear(960, n_classes)
  
    def forward(self, input_ids, attention_mask, features, labels=None):
#         print(input_ids.shape)
#         print(attention_mask.shape)
#         print(features.shape)
#         print(labels.shape)
        last_hidden_state, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False)

        cls = last_hidden_state[:,0,:]
        pooled_output = self.f1(self.drop(pooled_output))
        stacked_layers = torch.hstack([cls, pooled_output])
#         print(pooled_output.shape)
        feats = self.f2(features)
        
        stacked_features = torch.hstack([stacked_layers, feats])
#         print(stacked_features.shape)
        logits = self.out(self.drop(stacked_features))
        

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.n_classes), labels.view(-1))

        output = (logits,)
        return ((loss,) + output) if loss is not None else output

In [18]:
model = SentimentClassifier(N_LABELS)

# заморозим некоторые слои
freeze_layers = list(range(5))
for layer_id in freeze_layers:
    for param in list(model.bert.encoder.layer[layer_id].parameters()):
        param.requires_grad = False
        

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
 # model = GPTClassifier(model_config, N_LABELS)

In [20]:
cd /kaggle/working

/kaggle/working


In [21]:
training_args = TrainingArguments(
    output_dir=f'checkpoints/gpt', #{run_base_dir}',
    overwrite_output_dir=True,
    do_train=True, do_eval=True,
    evaluation_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-5,
    weight_decay=1e-1,
    num_train_epochs=8,
    warmup_ratio=0.1, save_strategy='epoch',
    save_total_limit=1, seed=42, fp16=True,
    dataloader_num_workers=1, group_by_length=True,
    report_to='none', load_best_model_at_end=True, metric_for_best_model='eval_acc',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
#                 optimizers=(optimizer, scheduler)
)

Using amp half precision backend


In [22]:
train_result = trainer.train()
print('train', train_result.metrics)

The following columns in the training set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `SentimentClassifier.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 129066
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 258136
/opt/conda/lib/python3.7/site-packages/transformers/trainer.py:1445: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the de

Epoch,Training Loss,Validation Loss,Acc,Micro F1,Macro F1
1,0.727200,0.566788,0.733485,0.733485,0.733392
2,0.507700,0.491642,0.760123,0.760123,0.758955
3,0.482500,0.525634,0.770304,0.770304,0.769570
4,0.499600,0.658723,0.771931,0.771931,0.770638
5,0.594800,0.804726,0.780345,0.780345,0.780184
6,0.550400,1.091265,0.781182,0.781182,0.781122
7,0.563400,1.264223,0.781600,0.781600,0.781599
8,0.547200,1.371004,0.780810,0.780810,0.780810


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `SentimentClassifier.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 21511
  Batch size = 4
Saving model checkpoint to checkpoints/gpt/checkpoint-32267
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in checkpoints/gpt/checkpoint-32267/tokenizer_config.json
Special tokens file saved in checkpoints/gpt/checkpoint-32267/special_tokens_map.json
/opt/conda/lib/python3.7/site-packages/transformers/trainer.py:1445: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old 

train {'train_runtime': 23894.0897, 'train_samples_per_second': 43.213, 'train_steps_per_second': 10.803, 'total_flos': 0.0, 'train_loss': 1.4684930664785079, 'epoch': 8.0}


In [23]:
dev_predictions = trainer.predict(test_dataset=tokenized_splits['validation'])
print('dev', dev_predictions.metrics)

The following columns in the test set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `SentimentClassifier.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 21511
  Batch size = 4


dev {'test_loss': 1.2642227411270142, 'test_acc': 0.7816001115708242, 'test_micro_f1': 0.7816001115708242, 'test_macro_f1': 0.7815988839224162, 'test_runtime': 175.2691, 'test_samples_per_second': 122.731, 'test_steps_per_second': 30.684}
